In [ ]:
# Gerekli kütüphaneleri yükleyin
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder

In [ ]:
# 1. Veri Yükleme ve Ön İşleme
# Örnek veri yükleniyor (Gerçek veri yolunu belirtin)
data = pd.read_csv("/content/sample_data/csv_building_structure.csv")  # Veri setini uygun dosya yolundan yükleyin

In [ ]:
data = data.dropna(subset=['damage_grade'])

In [ ]:
data['damage_grade'] = data['damage_grade'].str[-1].astype(int)

In [ ]:
data['technical_solution_proposed'].unique()

array([0])

In [ ]:
pd.set_option("display.max_columns", None)
data

,building_id,district_id,vdcmun_id,ward_id,count_floors_pre_eq,count_floors_post_eq,age_building,plinth_area_sq_ft,height_ft_pre_eq,height_ft_post_eq,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,condition_post_eq,damage_grade,technical_solution_proposed
0,120101000011,12,1207,120703,1,1,9,288,9,9,Flat,Other,Bamboo/Timber-Light roof,Mud,Not applicable,Not attached,Rectangular,0,1,0,0,0,0,0,1,0,0,0,Damaged-Used in risk,3,0
1,120101000021,12,1207,120703,1,1,15,364,9,9,Flat,Other,Bamboo/Timber-Light roof,Mud,Not applicable,Not attached,Rectangular,0,1,0,0,0,0,0,1,0,0,0,Damaged-Repaired and used,5,3
2,120101000031,12,1207,120703,1,1,20,384,9,9,Flat,Other,Bamboo/Timber-Light roof,Mud,Not applicable,Not attached,Rectangular,0,1,0,0,0,0,0,0,0,0,0,Damaged-Repaired and used,2,1
3,120101000041,12,1207,120703,1,1,20,312,9,9,Flat,Other,Bamboo/Timber-Light roof,Mud,Not applicable,Not attached,Rectangular,0,1,0,0,0,0,0,0,0,0,0,Damaged-Repaired and used,2,1
4,120101000051,12,1207,120703,1,1,30,308,9,9,Flat,Other,Bamboo/Timber-Light roof,Mud,Not applicable,Not attached,Rectangular,0,1,0,0,0,0,0,0,0,0,0,Damaged-Repaired and used,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762101,366709001241,36,3603,360302,2,0,60,165,18,0,Flat,Mud mortar-Stone/Brick,Bamboo/Timber-Heavy roof,Mud,Timber-Planck,Attached-2 side,Rectangular,0,1,0,0,0,0,0,0,0,0,0,Damaged-Rubble clear,5,3
762102,366709001251,36,3603,360302,2,0,35,342,18,0,Flat,Mud mortar-Stone/Brick,Bamboo/Timber-Heavy roof,Mud,Timber-Planck,Attached-1 side,Rectangular,0,1,0,0,0,0,0,0,0,0,0,Damaged-Rubble clear,5,3
762103,366709001261,36,3603,360302,2,0,35,342,18,0,Flat,Mud mortar-Stone/Brick,Bamboo/Timber-Heavy roof,Mud,Timber-Planck,Not attached,Rectangular,0,1,0,0,0,0,0,0,0,0,0,Damaged-Rubble clear,5,3
762104,366709001271,36,3603,360302,2,0,19,306,18,0,Flat,Mud mortar-Stone/Brick,Bamboo/Timber-Heavy roof,Mud,Timber-Planck,Not attached,Rectangular,0,1,0,0,0,0,0,0,0,0,0,Damaged-Rubble clear,5,3


In [ ]:
# Gerekli sütunları seçin (örnek değişkenler)
numerical_features = ['age_building', 'plinth_area_sq_ft', 'height_ft_pre_eq']
categorical_features = ['land_surface_condition', 'foundation_type', 'ground_floor_type',
                        'roof_type', 'plan_configuration', 'position']

target_damage = 'damage_grade'  # Hasar derecesi hedef değişkeni
target_repair = 'technical_solution_proposed'  # Onarım stratejisi hedef değişkeni

In [ ]:
# Kategorik değişkenleri one-hot encoding ile dönüştürme
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

In [ ]:
le = LabelEncoder()
data['technical_solution_proposed'] = le.fit_transform(data['technical_solution_proposed'])

print(le.classes_)

['Major repair' 'Minor repair' 'No need' 'Reconstruction']


In [41]:
X = preprocessor.fit_transform(data)
y_damage = data[target_damage] - 1
y_repair = data[target_repair] - 1


# 2. Sınıf Dengesizliği için SMOTE Kullanımı
smote = SMOTE(random_state=42)
X_resampled_damage, y_resampled_damage = smote.fit_resample(X, y_damage)
X_resampled_repair, y_resampled_repair = smote.fit_resample(X, y_repair)

In [43]:
# 3. Veri Setini Eğitim ve Test Olarak Bölme
X_train_d, X_test_d, y_train_d, y_test_d = train_test_split(X_resampled_damage, y_resampled_damage, test_size=0.33, random_state=42)
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_resampled_repair, y_resampled_repair, test_size=0.33, random_state=42)

# 4. XGBoost ile Model Eğitimi ve Tahmin
# Hasar derecesi tahmini
xgb_damage = XGBClassifier(random_state=42)
xgb_damage.fit(X_train_d, y_train_d)
y_pred_d = xgb_damage.predict(X_test_d)

y_train_r=y_train_r+1

# Onarım stratejisi tahmini
xgb_repair = XGBClassifier(random_state=42)
xgb_repair.fit(X_train_r, y_train_r)
y_pred_r = xgb_repair.predict(X_test_r)

In [44]:
# 5. Performans Değerlendirmesi
print("Hasar Derecesi Tahmini Raporu")
print(classification_report(y_test_d, y_pred_d))
print("Doğruluk Skoru:", accuracy_score(y_test_d, y_pred_d))

print("\nOnarım Stratejisi Tahmini Raporu")
print(classification_report(y_test_r, y_pred_r))
print("Doğruluk Skoru:", accuracy_score(y_test_r, y_pred_r))

Hasar Derecesi Tahmini Raporu
              precision    recall  f1-score   support

           0       0.58      0.69      0.63     91080
           1       0.41      0.30      0.35     90866
           2       0.38      0.16      0.22     91183
           3       0.35      0.20      0.25     91196
           4       0.39      0.83      0.53     90689

    accuracy                           0.43    455014
   macro avg       0.42      0.43      0.40    455014
weighted avg       0.42      0.43      0.40    455014

Doğruluk Skoru: 0.43399323976844667

Onarım Stratejisi Tahmini Raporu


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00    155692
           0       0.32      0.23      0.26    155156
           1       0.24      0.13      0.17    154947
           2       0.01      0.02      0.01    154895
           3       0.00      0.00      0.00         0

    accuracy                           0.09    620690
   macro avg       0.12      0.07      0.09    620690
weighted avg       0.14      0.09      0.11    620690

Doğruluk Skoru: 0.09280155955468913


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
